# Linear Mixed Effects Models (LMEM)

## Introduction

The traditional regression models assume independence between observations in the dataset. However, this assumption can be unrealistic IRL since we may have multiple observations per person over time, or observations that are obtained from a group of people that share characteristics that may influence the outcome. Here we can not use a linear or logistic regression to model the outcome. That's where mixed effects/hierarchical/multilevel/varying coefficient models come in. 

The random effects model where coefficients vary across clusters is

$$
Y_{ij} = \beta_{0i} + \beta_{1}X_{ij} + \epsilon_{ij}
$$

In this model, i represents the cluster, j represents the observation within cluster, and $Y_{ij}$ is the outcome for cluster i's observation j. 

Also, for this specic model, the intercept was specified for each of the cluster and can be rewritten as 

$$
\beta_{0i} = \beta_0 + u_i
$$

- $\beta_0$ is the part of the intercept common to all clusters
- $u_i$ is the part of the intercept specific to each cluster. This means that this the random variation in the outcome that is not explained by $\beta_0$
    - $u_i$ is assumed to be random, therefore the name "random effect model"
    - It is also assumed to follow a normal distribution distribution with unknown variance
    
    $$
    u_i \sim N(0, \sigma^2_{u})
    $$

The traditional linear regression model can be seen as a special casse of the mixed model where $u_i$ = 0 for all i

Therefore, the random intercept model above, can be rewritten as follows for clarity: 

$$
Y_{ij} = \beta_{0} + u_i + \beta_{1}X_{ij} + \epsilon_{ij}
$$

- $\epsilon_{ij}$ are the errors of the model and represent the random variation in the outcome that is not not explained by both the fixed and random effects. They are all assumed to be independent of each other. WHY? THEY CAN ALSO BE DEPENDENT? They are also assumed to follow a normal distribution with mean zero and unknown variance.

$$
\epsilon_{ij} \sim N(0, \sigma^2_{\epsilon})
$$

And finally, $\epsilon_{ij}$ and $u_i$ are independent of each other. 

### Independence between $\epsilon_{ij}$ and $u_i$

The random variation in the response variable that is not explained by the fixed and random effects ($\epsilon_{ij}$) is not related to the random variation among the clusters ( $u_i$). This means that knowing the value of one for a specific cluster $i$ does not provide any information about the size or the direction of the other.  This allows us to estimate the variances of the random effects and the residual errors separately, which is necessary for accurate inference

Expectation of the outcome is the same as in the traditional linear regression model where

$$
E[Y_{ij}|X_{ij}] = \beta_0 + \beta_1X_{ij}
$$

while the varianc of the oucome, which is **marginal** over $u_i$ is

$$
Var[Y_{ij}|X_{ij}] = Var[u_i] + Var[\epsilon_{ij}] \\
                    = \sigma^2_{u} + \sigma^2_{\epsilon}
$$
    - The variability of the observations is due to both the variability between clusters and the varition that is not explained by the model ($\epsilon_{ij}$)
    
### Covariance between observations from the same cluster

$$
Cov(Y_{ij}, Y_{ik}) = Cov(u_i + \epsilon_{ij}, u_i + \epsilon_{ik}) \\
                    = Cov(u_i, u_i) + Cov(u_i, \epsilon_{ij}) + Cov(u_i, \epsilon_{ik}) + Cov(\epsilon_{ij}, \epsilon_{ik})
                    = Var(u_i) + 0 + 0 + 0
                    = \sigma^2_{u}
$$

This means that the correlation between two observations from the same cluster is not zero!! This is why we need a mixed model

- This model describes the correlation between subjects in the same cluster
- This is a random intercept model and it has the correlation that is constant for all pairs of observations within the clusters. May not be appropriate for some data (e.g., longitudinal data where we assume autocorrelation)
- Under the traditional regression model, $u_i = 0$, implying that the observations within the same cluster are independent


NOTE: VERIFY IF COVARIANCE BETWEEN CLUSTERS IS ZERO!

The model above (random intercept) can be modified to include a cluster-specific slope $\beta_{1i} = \beta_1 + u_1i$: 

$$
Y_{ij} = \beta_{0i} + \beta_{1i}X_{ij} + \epsilon_{ij}
$$

Assumptions: 
- $\epsilon_{ij}$ are all independent with $N(0, \sigma^2_{\epsilon}$
- u's are all independent
- $\epsilon$'s are independent from u's
- us can be set to be independent to each other or allowed to be correlated


### Expectation, variance and covariance random interecept

- Expectation
$$
E(Y_ij|X_ij) = \beta_0 + \beta_1*X_{ij}
$$

- Variance
$$
Var[Y_{ij}|X_ij] = \sigma^2_{u0} + \sigma^2_{u1}X^2_{ij} + \sigma^2_{\epsilon}
$$

This assumes independence between u's. it is complicated otherwise

- Covariance within clusters (again, u's are independent in this case)

$$
Cov(Y_{ij}, Y_{ik}) = \sigma^2_{u0} + X_{ij}X_{ik}\sigma^2_{u1}
$$

- Covariance betwen clusters
$$
Cov(Y_{ij}, Y_{i'k}) = 0
$$



- If we allow coefficients to vary, we may have correlations between random coefficients

If $Cov(u_{0i}, u_{1i} $ < 0 suggests that if the intercept is low, the slope will be high and if it is positive it suggests that if the intercept is high, the slope will also be high.

- We can also add more covariates in the model


- $\beta$'s are called fixed coefficients
- $u$'s are random effects
- $\epsilon$'s are errors

The model above can be summarized as

$$
Y_{ij} = X\beta + Zu + \epsilon
$$

where $X$ is the design matrix of the fixed effects, $\beta$ is the vector of fixed effects, $Z$ is the design matrix of random effects, $u$ is the vector of random effects, and $\epsilon$ is the error that is not accounted for by fixed and random effect. We want to estimate $\beta$'s and $u$'s in the model

## Example

In [1]:
# load libraries
library(tidyverse)
library(data.table)

── Attaching packages ────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.0     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 1.0.0
── Conflicts ───────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attachement du package : ‘data.table’


Les objets suivants sont masqués depuis ‘package:dplyr’:

    between, first, last


L'objet suivant est masqué depuis ‘package:purrr’:

    transpose




In [3]:
# load data
exam = read_csv("./lmedata/exam.csv", show_col_types = FALSE)
setDT(exam)
head(exam)

obs,school,normexam,schgend,schavg,vr,intake,standLRT,sex,type,student
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,1,0.2613242,mixed,0.1661752,mid 50%,bottom 25%,0.6190592,F,Mxd,143
2,1,0.1340672,mixed,0.1661752,mid 50%,mid 50%,0.2058022,F,Mxd,145
3,1,-1.7238820,mixed,0.1661752,mid 50%,top 25%,-1.3645760,M,Mxd,142
4,1,0.9675862,mixed,0.1661752,mid 50%,mid 50%,0.2058022,F,Mxd,141
5,1,0.5443412,mixed,0.1661752,mid 50%,mid 50%,0.3711052,F,Mxd,138
6,1,1.7348992,mixed,0.1661752,mid 50%,bottom 25%,2.1894372,M,Mxd,155


In [10]:
exam$school  <- as.factor(exam$school )

In [4]:
dim(exam)

[1] 4059   11

In [5]:
uniqueN(exam$school)

[1] 65

- 4059 students in 65 schools
- Exam scores - one exam per student
- Correlation - If there are “high-performing” schools, maybe kids in
these schools are more alike
- Outcome: 16 year reading test score
- Covariates:
    - Sex
    - 11 year reading test score
    - School sex (mixed, boys, girls)
    - School average score
- All scores are standardized

## Fit model with just standLTR as a predictor

In [ ]:
def get_Vi(Zi, D, Ri):
    """
    Computes the variance-covariance structure of the repeated
    observations within cohort i
    """
    Vi = np.matmul(Zi, D)
    Vi = np.matmul(Vi, np.transpose(Zi)) + Ri
    return Vi
    

In [13]:
# define variance-covariance structure

get_Vi  <- function(Zi, D, Ri){
    Vi  <- Zi %*% D
    Vi  <- Vi %*% t(Zi) + Ri
    
    return(Vi)
}


get_MLE_beta  <- function(Xi_list, yi_list, Vi_list){
    
    p = dim(Xi_list[0])[2]
    
}




def get_mle_beta(Xi_list, yi_list, Vi_list):
    """
    Computes the MLE for beta
    """
    p = Xi_list[0].shape[1]
    
    mat1 = np.zeros((p, p))
    mat2 = np.zeros(p)
    
    for i in range(len(Xi_list)):
        e
        Xi = Xi_list[i]
        yi = yi_list[i]
        Vi = Vi_list[i]
        
        m1 = np.matmul(np.transpose(Xi), np.linalg.inv(Vi))
        m1 = np.matmul(m1, Xi)
        
        m2 = np.matmul(np.transpose(Xi), np.linalg.inv(Vi))
        m2 = np.matmul(m2, yi)
        
        mat1 += m1
        mat2 += m2
        
    # MLE estimates for beta
    beta = np.matmul(np.linalg.inv(mat1), mat2)
    return beta




# specify design matrix 
X  <-  model.matrix(~1+standLRT, data = exam) # fixed effects design matrix
Z  <-  model.matrix(~1+school, data = exam) # random effects design matrix
y  <- exam$normexam

# model formula
formula  <- y ~ X %*% beta + Z %*% u

# specify initial values for fixed and random effects
beta  <- rep(0, ncol(X))
u  <- rnorm(nlevels(exam$school))

# define the log-likelihood function
logLikelihood  <- function(beta, u) {
    theta  <- X %*% beta + Z %*% u
    sigma  <- exp(theta)
    -sum(dnorm(y, theta, sigma, log = TRUE))
}

# maximize the log-likelihood function
fit  <- optim(c(beta, u), logLikelihood, method = "L-BFGS-B", 
              lower = c(-Inf, rep(-Inf, nlevels(exam$school))), 
              upper = c(Inf, rep(Inf, nlevels(exam$school))))


ERROR: Error in X %*% beta: arguments inadéquats


In [ ]:

# Specify the design matrices and vectors
X <- model.matrix(~1+Days, data = sleepstudy)  # Fixed effects design matrix
Z <- model.matrix(~1+Subject, data = sleepstudy)  # Random effects design matrix
y <- sleepstudy$Reaction  # Response vector

# Specify the model formula
formula <- y ~ X %*% beta + Z %*% u

# Specify the initial values for the fixed and random effects
beta <- rep(0, ncol(X))
u <- rnorm(nlevels(sleepstudy$Subject))

# Define the log-likelihood function
logLikelihood <- function(beta, u) {
  eta <- X %*% beta + Z %*% u
  sigma <- exp(theta)
  -sum(dnorm(y, eta, sigma, log = TRUE))
}

# Maximize the log-likelihood function
fit <- optim(c(beta, u), logLikelihood, method = "L-BFGS-B", lower = c(-Inf, rep(-Inf, nlevels(sleepstudy$Subject))), upper = c(Inf, rep(Inf, nlevels(sleepstudy$Subject))))

# Extract the coefficients
beta_hat <- fit$par[1:ncol(X)]
u_hat <- fit$par[(ncol(X)+1):length(fit$par)]